In [23]:
import helpers
import json
import wowsims

Combinations of gear to try:

1. 2p t8 permutations: Head/Shoulder, Head/Chest, Head/Gloves, Head/Legs, Chest/Gloves, Chest/Legs, Chest/Gloves, Gloves/Legs
2. 2p t9 permutations: ^^

In [24]:
t8_slot_ids = [(0, 46161), (2, 46157), (4, 46159), (6, 46158), (8, 46160)]
t9_slot_ids = [(0, 48201), (2, 48198), (4, 48199), (6, 48202), (8, 48200)]
non_set_slot_ids = [(0, 47688), (1, 47060), (2, 47708), (3, 47545), (4, 47945), (5, 45611), (6, 47945), (7, 47112), (8, 45536), (9, 47077), (10, 47075), (11, 45608), (12, 45931), (13, 47131), (14, 47491), (16, 47668)]

In [26]:
t8_4pc = list(permute(t8_slot_ids, 4))
t9_4pc = list(permute(t9_slot_ids, 4))
t9_1pc = list(permute(t9_slot_ids, 1))
t8_2pc = list(permute(t8_slot_ids, 2))
t9_2pc = list(permute(t9_slot_ids, 2))
t8_t9_4pc = []
for t8_pcs in t8_2pc:
    for t9_pcs in t9_2pc:
        if set([p[0] for p in t8_pcs]).intersection(set([p[0] for p in t9_pcs])):
            continue
        else:
            t8_t9_4pc.append(t8_pcs + t9_pcs)
sets_to_consider = t8_t9_4pc + t8_4pc + t9_4pc + t8_2pc + t9_2pc + t9_1pc

full_sets_to_consider = []
for item_set in sets_to_consider:
    altered_set = non_set_slot_ids.copy()
    replacements = dict(item_set)
    for index, item in enumerate(non_set_slot_ids):
        if item[0] in replacements:
            altered_set[index] = (item[0], replacements[item[0]])
    full_sets_to_consider.append(altered_set)

base_settings = json.load(open('data/p2_feral.json'))
del base_settings['raid']['parties'][0]['players'][0]['database']

In [27]:
results = []
url = "https://wowsims.github.io/wotlk/feral_druid/#"
for item_set in full_sets_to_consider:
    settings = json.loads(json.dumps(base_settings))
    helpers.apply_item_set(settings, item_set)
    helpers.apply_gem_policy(settings)
    result = wowsims.runSim(settings)
    link_hash = wowsims.encodeSettings(settings)
    dps = result['raidMetrics']['dps']['avg']
    results.append((dps, url+link_hash))
results.sort(key=lambda x: x[0], reverse=True)

results[:5]

[(12207.051860824164,
  'https://wowsims.github.io/wotlk/feral_druid/#eJzsVDFoFEEUnfdvb91dlIwh4HjmjnVAiIHA7KwHWsjtWVke2NgIJ2gqC7FSq1MRgrYqeDYiFgGLgBjJFkeEaOIVHiHoFdoEPSVisQdR2U7ZWcVaTDpf8dn33r7HwAzf2+eQosNoUBPn0QJmgVVgHdgARlkpKTo2R6kPb4R3UxKfy9J+ukSdmDyXv05IFuZj8nby5ZTEelkWOobFmyRuj+feCF9JSaRlac/H1DP2owGJ6xVZ6Bm7+53ErXFj5//PDEm6PcNy4WVKIskL+j8y4WNC4s6fiMs//D6Jy2cHvz4t/uVTNu8Ns8i1TRKvytKei2kuJo95Fn/yleQuh5TVsE5aFzEDXLYcCLoKu3Hu9KWzF27g4FT1UKiVDkIdaq0CpaphoJUKVRioahBMaRVWVagC3QZ7C/YewlleJP5wQOLFIvntNzSxskZqaY02YA2x4wF9I+8Zsf/4J4yqv008xoHsrXOIoo8JHGXMr0fF45VTjDXrLVRKNj8j4EPeLLgm0JqOdufRsWhv+26Gbm3/tEG/pt+RKPonJrf5KodXoq2oaXWfb0nPduCImZ0kquc8rTWRa2y1NllwFvYcG3PgU5Tvp/tmRS0APwMAAP//Xn6OMA=='),
 (12190.114654999254,
  'https://wowsims.github.io/wotlk/feral_druid/#eJzsVEFrE0EUnu/tZt1dlIyh4BizYR0QaqEwu2tAD5KNJ48BL16ECNqTB/GknqIiFL2qYLyIeCh4KIiR7iFUqLZEsJSiOegl2CgVDxuowt6U3dUfILa3fgyP+d6b7+PBPJ59yCRFx9GkFi6jDcwBa8AQ2ARKrBwXTIOjPIBd5P2ExDdHGi+XqBeRbfH3MUmtG5G9ly8nJIaO1HoZi7ZI3K/ktSJfSUgkjjS6EeXl

In [34]:
from IPython.core.display import display, HTML
for result in results:
    display(HTML(f'<a href="{result[1]}">{result[0]:.0f}</a>'))
    print(result[0], result[1])

12207.051860824164 https://wowsims.github.io/wotlk/feral_druid/#eJzsVDFoFEEUnfdvb91dlIwh4HjmjnVAiIHA7KwHWsjtWVke2NgIJ2gqC7FSq1MRgrYqeDYiFgGLgBjJFkeEaOIVHiHoFdoEPSVisQdR2U7ZWcVaTDpf8dn33r7HwAzf2+eQosNoUBPn0QJmgVVgHdgARlkpKTo2R6kPb4R3UxKfy9J+ukSdmDyXv05IFuZj8nby5ZTEelkWOobFmyRuj+feCF9JSaRlac/H1DP2owGJ6xVZ6Bm7+53ErXFj5//PDEm6PcNy4WVKIskL+j8y4WNC4s6fiMs//D6Jy2cHvz4t/uVTNu8Ns8i1TRKvytKei2kuJo95Fn/yleQuh5TVsE5aFzEDXLYcCLoKu3Hu9KWzF27g4FT1UKiVDkIdaq0CpaphoJUKVRioahBMaRVWVagC3QZ7C/YewlleJP5wQOLFIvntNzSxskZqaY02YA2x4wF9I+8Zsf/4J4yqv008xoHsrXOIoo8JHGXMr0fF45VTjDXrLVRKNj8j4EPeLLgm0JqOdufRsWhv+26Gbm3/tEG/pt+RKPonJrf5KodXoq2oaXWfb0nPduCImZ0kquc8rTWRa2y1NllwFvYcG3PgU5Tvp/tmRS0APwMAAP//Xn6OMA==
12190.114654999254 https://wowsims.github.io/wotlk/feral_druid/#eJzsVEFrE0EUnu/tZt1dlIyh4BizYR0QaqEwu2tAD5KNJ48BL16ECNqTB/GknqIiFL2qYLyIeCh4KIiR7iFUqLZEsJSiOegl2CgVDxuowt6U3dUfILa3fgyP+d6b7+PBPJ59yCRFx9GkFi6jDcwBa8AQ2ARKrBwXTIOjPIBd5P2ExDdHGi+XqBeRbfH3MUmtG5G9ly8nJIaO1HoZi7ZI3K/ktSJfSUgkjjS6EeXlZyMSt6tSW83Y24T